In [1]:
import numpy as np
from keras.datasets import mnist
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

2024-03-25 14:27:33.339139: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-25 14:27:33.339258: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-25 14:27:33.500131: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
(x_train_inp , y_train_inp ), (x_test_inp , y_test_inp)  = mnist.load_data()
x_train_inp = x_train_inp/255.0
converted_ans = []

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
for i in range(len(y_train_inp)):
    ans = np.zeros(10)
    ans[y_train_inp[i]] = 1
    converted_ans.append(ans)

In [4]:
x_train , x_test , y_train , y_test = train_test_split(x_train_inp,converted_ans,test_size = 0.35,shuffle  = True,random_state = 42)

In [5]:
class DigitModel:
    total_layers = []
    output_layers = []
    learning_rate = 0.0
    
    def __init__(self,lr):
        self.total_layers = []
        self.learning_rate = lr
    
    def add_layer(self,size_of_input_layer,number_of_neurons):
        weights_layer = np.random.randn(size_of_input_layer , number_of_neurons) 
        weights_layer = np.transpose(weights_layer)
        self.total_layers.append(weights_layer)
        
    def activation_sigmoid(self,x):
        return 1/(1 + np.exp(-x))

    def sigmoid_der(self,x):
        return (1 - x ) * x
    
    def categorical_cross_entropy_loss(self,predict, real):
        # Ensure predictions are probabilities by applying softmax
        predict = self.softmax(predict)
    
        # Clip values to avoid log(0) issues
        predict = np.clip(predict, 1e-15, 1 - 1e-15)
    
        # Compute cross-entropy loss
        loss = -np.sum(real * np.log(predict))
    
        return loss

    def softmax(self,x):
        exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

    def softmax_derivative(self,x):
        s = self.softmax(x)
        return s * (1 - s)

    def categorical_cross_entropy_loss_derivative(self,predict, real):
        # Derivative of cross-entropy loss with respect to softmax input
        d_loss = predict - real
        
        return d_loss

    def loss_function(self,predict,real ):
        print(real)
        sum = np.sum((real - predict) ** 2)
        return sum

    def predict_final(self,final_out):
        max = -1000
        index = -1
        for i in range(0,10):
            if (final_out[i] > max):
                max = final_out[i]
                index = i
        return index

    def predict_output(self,input_arr):
        prev = input_arr
        for layer in self.total_layers:
            prev = self.activation_sigmoid(np.dot(layer,prev))
            self.output_layers.append(prev)
        if (self.output_layers[-1].sum() != 0.0 and self.output_layers[-1].min() != 0 ):
            self.output_layers[-1] = self.output_layers[-1]/self.output_layers[-1].min()
            self.output_layers[-1] = self.output_layers[-1]/self.output_layers[-1].sum() 

            
            
            
            
    def backpropagation(self,input_data ,expected_output):
        self.output_layers.insert(0,input_data)
        prev = []
        chage = []
        for i in range(len(self.output_layers)-1,0,-1):
             if len(self.output_layers)-1 == i:
                change = self.categorical_cross_entropy_loss_derivative(self.output_layers[i],expected_output) * self.sigmoid_der(self.output_layers[i])
                #  change = 2*(self.output_layers[i]-expected_output) * self.sigmoid_der(self.output_layers[i])
                weight_op_update = self.learning_rate * np.outer(change, self.output_layers[i-1])
                self.total_layers[i-1] -= weight_op_update
             else:
                 change = np.dot(prev,self.total_layers[i])*self.sigmoid_der(self.output_layers[i])
                 weight_hl_update = self.learning_rate * np.outer(change,self.output_layers[i-1])
                 self.total_layers[i-1] -= weight_hl_update
             prev = change

            

    def fit(self,x_train,y_train,epochs):
        loss_overall = 0
        count = 0
        total = 0
        for i in range(epochs):
            ec = 0
            et = 0
            loss_sum = 0
            for j in range(len(x_train)):
                input_arr = np.reshape(x_train[j],784)
                self.predict_output(input_arr)
                loss_sum += self.categorical_cross_entropy_loss(self.output_layers[-1],y_train[j])
                if(self.predict_final(self.output_layers[-1]) == self.predict_final(y_train[j])):
                    count = count + 1
                    ec = ec + 1
                self.backpropagation(input_arr,y_train[j])
                self.output_layers = []
                total = total + 1
                et = et + 1
            print("Epoch Loss : ",loss_sum/len(x_train))
            print("Epoch Accuracy : ", (ec/et)*100.0)
            loss_overall = loss_overall + loss_sum
        accuracy = (count / total) * 100.0 
        print("Final Loss : ",loss_overall/epochs)
        print("Final accuracy : ",accuracy) 
        
    def evaluate(self,x_test,y_test):
        count = 0
        loss_sum = 0
        total = 0
        for j in range(len(x_test)):
                input_arr = np.reshape(x_test[j],784)
                self.predict_output(input_arr)
                loss_sum += self.categorical_cross_entropy_loss(self.output_layers[-1],y_test[j])
                if(self.predict_final(self.output_layers[-1]) == self.predict_final(y_test[j])):
                    count = count + 1
                self.output_layers = []
                total = total + 1
        print("Evaluation Loss : ",loss_sum/len(x_test))
        accuracy = (count / total) * 100.0 
        print("Evaluation Accuracy : ",accuracy)

In [6]:
size_of_input_layer = 784
size_of_output_layer = 10

In [7]:
digits = DigitModel(0.02)

digits.add_layer(size_of_input_layer,100)
digits.add_layer(100,100)
digits.add_layer(100,size_of_output_layer)


In [8]:
digits.fit(x_train,y_train,50)

Epoch Loss :  2.062577465036248
Epoch Accuracy :  48.243589743589745
Epoch Loss :  1.7876423641776666
Epoch Accuracy :  76.67948717948718
Epoch Loss :  1.7142047151393378
Epoch Accuracy :  82.4076923076923
Epoch Loss :  1.6781286256800738
Epoch Accuracy :  85.14871794871794
Epoch Loss :  1.655243039054073
Epoch Accuracy :  86.85641025641026
Epoch Loss :  1.638790172443864
Epoch Accuracy :  88.12564102564102
Epoch Loss :  1.6260330823781353
Epoch Accuracy :  89.08205128205128
Epoch Loss :  1.6156388049513197
Epoch Accuracy :  89.90769230769232
Epoch Loss :  1.6069665700953901
Epoch Accuracy :  90.54102564102564
Epoch Loss :  1.5995721196323873
Epoch Accuracy :  91.16923076923077
Epoch Loss :  1.5931622768950742
Epoch Accuracy :  91.66666666666666
Epoch Loss :  1.5875369786994682
Epoch Accuracy :  92.13076923076923
Epoch Loss :  1.58253411303974
Epoch Accuracy :  92.51794871794871
Epoch Loss :  1.5780360107488471
Epoch Accuracy :  92.77948717948718
Epoch Loss :  1.5739636317350756
Epoch 

In [9]:
digits.evaluate(x_test,y_test)

Evaluation Loss :  1.5495044162430591
Evaluation Accuracy :  92.92380952380952
